# Amortisierte Analyse

Nicht immer ist die Angabe der worst-case Komplexität hilfreich. Was ist nämlich, wenn der worst-case nur sehr selten eintritt, aber in den meisten Fällen eine wesentlich bessere Laufzeit erzielt? Dies ist z.B. beim Hinzufügen von Elementen an eine Array-List der Fall. In den meisten Fällen geht dies problemlos mit einem Aufwand von $\mathcal{O}(1)$, doch ist das Array nicht lang genug, so müssen alle Werte in eine neues Array kopiert werden, was zu einem Aufwand von $\mathcal{O}(n)$ führt. Dadurch liegt die worst-case Komplexität bei $\mathcal{O}(n)$, was zunächst ziemlich schlecht zu sein scheint, da andere Datenstrukturen dies (selbst im worst-case) in $\mathcal{O}(1)$ können. Um hier eine genaue Aussage treffen zu können, bedarf es der amortisierten Analyse.

Die amoritsierte Analyse (amortized analysis) berechnet die mittleren Kosten über einer Folge von Operationen, in der viele dieser Operationen billig sind und nur wenige teuer in Bezug auf deren Beitrag zur Gesamtzeit.

## Aggregat-Analyse

Bei der Aggregat-Analyse gilt:

$$T(n)_{amort} = \frac{\text{Summe der Kosten aller Operationen}}{\text{Anzahl der Operationen}}$$

Die Kosten der Operationen $1, 2, ..., n$ wird durch die Anzahl $n$ der Operationen geteilt.

### Array-List

Bei einer Array-List wird zunächst ein kleines Array genommen. Sobald der Platz nicht mehr ausreicht und man das $n+1$. Element einfügen möchte, so wird ein neues Array mit doppelter Länge erstellt und die bisherigen Werte reinkopiert. Sollte dieses nicht merhe ausreich, so wird auch dieses wieder verdoppelt.

$$
\begin{align}
T(n)_{gesamt} & = n + 1 + \dotsc + 1 + \frac{n}{2} + 1 + \dotsc + 1 + \frac{n}{4} + 1 + \dotsc + 1 + \frac{n}{8} + \dotsc + 1 \\
 & = \sum_{i=0}^{\infty}(n \cdot (\frac{1}{2})^i)) + (n - \log_2 n) \cdot 1 \\
 & = n \cdot \sum_{i=0}^{\infty}((\frac{1}{2})^i) + (n - \log_2 n) \cdot 1 \\
\end{align}
$$

Da es sich bei der Summe um eine geometrische Reihe handelt, kann sie gelöst werden.

$$
\begin{align}
T(n)_{gesamt} & = n \cdot \frac{1}{1-\frac{1}{2}} + (n - \log_2 n) \cdot 1 \\
 & = 2n + (n - \log_2 n) \cdot 1 \\
 & = 2n + \mathcal{O}(n) \\
 & = \mathcal{O}(n)
\end{align}
$$

$$
\begin{align}
T(n)_{amort} & = \frac{\mathcal{O}(n)}{n} \\
 & = \mathcal{O}(1)
\end{align}
$$

Damit ergeben sich für die Insert-Opertion bei einer Array-List amortisierte Kosten von $\mathcal{O}(1)$.

### Binärzähler

Situation: $n$ Inkrementierungen eines $k$-Bit-Binärzählers mit Anfangswert 0

Array: $A[0 \dotsc k-1], A[i] \in \{0,1\},$

$$x = \sum_{i=0}^{k-1} A[i] \cdot 2^i$$

Worst case: Alle $k$ Bits werden invertiert, da sie von 1 auf 0 wechseln. Dies ist beim Übergang von $2^i - 1$ zu $2^i$ der Fall. Folglich beträgt der Aufwand je Inkrementierungsschritt im schlechtesten Fall $\mathcal{O}(k) = \mathcal{O}(\log n)$. Dies ist zwar korrekt, aber zu grob.

In [22]:
import pandas as pd


def increment_cost(n):
    if n == 0:
        return 0
    return n ^ (n-1)


def total_costs(bin_strings):
    if len(bin_strings) == 0:
        return []
    total_costs_lst = total_costs(bin_strings[:-1])
    if len(total_costs_lst) == 0:
        total_cost = 0
    else:
        total_cost = total_costs_lst[len(total_costs_lst)-1]
    for digit in bin_strings[len(bin_strings)-1]:
        if digit == "1":
            total_cost += 1
    return total_costs_lst + [total_cost]
            
    
A_cost = []
max_n = 25
inc_costs = []

for i in range(0, max_n):
    inc_costs.append("{0:b}".format(increment_cost(i)))

for i in range(0, 7):
    A_cost.append(list())
    for j in range(0, max_n):
        A_cost[i].append(0)

for n in range(0, max_n):
    inc_cost = "{0:b}".format(increment_cost(n))
    for i in reversed(range(0, len(inc_cost))):
        A_cost[i][n] = inc_cost[i]
        
        
table = pd.DataFrame({'A[6]': pd.Series(A_cost[6]),
                     'A[5]': pd.Series(A_cost[5]),
                     'A[4]': pd.Series(A_cost[4]),
                     'A[3]': pd.Series(A_cost[3]),
                     'A[2]': pd.Series(A_cost[2]),
                     'A[1]': pd.Series(A_cost[1]),
                     'A[0]': pd.Series(A_cost[0]),
                    'Total costs': pd.Series(total_costs(inc_costs))}, 
                   columns=['A[6]', 'A[5]', 'A[4]', 'A[3]', 'A[2]', 'A[1]', 'A[0]', 'Total costs'])

print(table)

    A[6]  A[5] A[4] A[3] A[2] A[1] A[0]  Total costs
0      0     0    0    0    0    0    0            0
1      0     0    0    0    0    0    1            1
2      0     0    0    0    0    1    1            3
3      0     0    0    0    0    0    1            4
4      0     0    0    0    1    1    1            7
5      0     0    0    0    0    0    1            8
6      0     0    0    0    0    1    1           10
7      0     0    0    0    0    0    1           11
8      0     0    0    1    1    1    1           15
9      0     0    0    0    0    0    1           16
10     0     0    0    0    0    1    1           18
11     0     0    0    0    0    0    1           19
12     0     0    0    0    1    1    1           22
13     0     0    0    0    0    0    1           23
14     0     0    0    0    0    1    1           25
15     0     0    0    0    0    0    1           26
16     0     0    1    1    1    1    1           31
17     0     0    0    0    0    0    1       